# CNN介绍

In [1]:
# 这里重点说明一下 torch.nn 和 torch.nn.functional 有何区别，下面用两者中的 convl2d 来举例说明：
# torch.nn 是一个类，模型的参数 weight 和 bias 会自动注册，我们只需要给出其形状即可，weight 和 bias 会在后续网络的训练中自动更新优化
# torch.nn.functional 只是一个函数，我们需要传入具体的 weight 和 bias 值，后续两者是否会随则会反向传播进行优化要看两者是否被加入到了参数列表

In [2]:
import torch
import torch.nn.functional as F

In [3]:
input = torch.tensor([[1, 2, 0, 3, 1],
                      [0, 1, 2, 3, 1],
                      [1, 2, 1, 0, 0],
                      [5, 2, 3, 1, 1],
                      [2, 1, 0, 1, 1]])
kernel = torch.tensor([[1, 2, 1],
                       [0, 1, 0],
                       [2, 1, 0]])
# 打印输入张量和卷积核的形状
print(input.shape)  # 输出: torch.Size([5, 5])
print(kernel.shape)  # 输出: torch.Size([3, 3])


torch.Size([5, 5])
torch.Size([3, 3])


In [4]:
# torch.reshape
    # Args:
    #     input (Tensor): the tensor to be reshaped    想要变换的张量
    #     shape (tuple of int): the new shape          想要变换为的形状

# torch.nn.functional.conv2d
    # Args:
    #     input,           输入，必须得是 (minibatch,in_channels,iH,iW) 的张量
    #     weight,          卷积核（权重），必须得是 (out_channels,in_channels / groups,kH,kW) 的张量
    #     bias=None,       偏置值，必须是 1 * out_channels 形状的张量
    #     stride=1,
    #     padding=0,
    #     dilation=1,
    #     groups=1
# input: (5, 5) -> (1, 1, 5, 5) 表示 (batch_size, channels, height, width)
# kernel: (3, 3) -> (1, 1, 3, 3) 表示 (out_channels, in_channels, kernel_height, kernel_width)
input = torch.reshape(input, (1, 1, 5, 5))
kernel = torch.reshape(kernel, (1, 1, 3, 3))
print(input.shape)  # 输出: torch.Size([1, 1, 5, 5])
print(kernel.shape)  # 输出: torch.Size([1, 1, 3, 3])


torch.Size([1, 1, 5, 5])
torch.Size([1, 1, 3, 3])


In [5]:
# 使用 torch.nn.functional.conv2d 进行卷积操作
# 参数解释:
# input: 输入张量，形状为 (batch_size, in_channels, height, width)
# weight: 卷积核，形状为 (out_channels, in_channels / groups, kernel_height, kernel_width)
# stride: 步幅，控制卷积核移动的步长
# padding: 填充，控制边界的填充大小
output1 = F.conv2d(input, kernel, stride=1)  # 步幅为 1
print(output1)  # 输出卷积结果


tensor([[[[10, 12, 12],
          [18, 16, 16],
          [13,  9,  3]]]])


In [6]:
# 这里必须使用类型转换
# 使用 torch.tensor([...]) 创建 input 和 kernel 时，默认它们是 LongTensor（整型）,torch.ones 创建的默认是浮点类型
# 而 F.conv2d 要求所有输入（包括 input、kernel、bias）类型必须 一致，通常是 float32（torch.float）
# 虽然下面的示例采用了 conv2d_bias = conv2d_bias.long() 这样的转换，但是一般不推荐，这例子是做一个反面教材
# 一般更推荐将所有传入参数都转换为浮点数，因为这样会让内部计算更快，有如下两种方法：
# 方法1（更推荐）:在定义参数的时候直接明确为浮点数
    # input = torch.tensor([[1, 2, 0, 3, 1],
    #                       [0, 1, 2, 3, 1],
    #                       [1, 2, 1, 0, 0],
    #                       [5, 2, 3, 1, 1],
    #                       [2, 1, 0, 1, 1]], dtype=torch.float)

    # kernel = torch.tensor([[1, 2, 1],
    #                        [0, 1, 0],
    #                        [2, 1, 0]], dtype=torch.float)

# 方法2（不推荐）：
    # input = input.float()
    # kernel = kernel.float()
conv2d_bias = torch.ones(1)
conv2d_bias = conv2d_bias.long()
print(conv2d_bias)
print(conv2d_bias.shape)
output2 = F.conv2d(input, kernel, bias = conv2d_bias, stride = 1)
print(output2)

tensor([1])
torch.Size([1])
tensor([[[[11, 13, 13],
          [19, 17, 17],
          [14, 10,  4]]]])


In [7]:
# 关于 stride 参数
# stride 控制卷积核移动的步长，步长越大，输出张量的尺寸越小
output3 = F.conv2d(input, kernel, stride=2)  # 步幅为 2
print(output3)  # 输出卷积结果


tensor([[[[10, 12],
          [13,  3]]]])


In [8]:
# 关于 padding 参数
# padding 用于在输入张量的边界添加额外的值，通常是 0
# padding 的大小会影响输出张量的尺寸
output4 = F.conv2d(input, kernel, stride=1, padding=1)  # 填充大小为 1
print(output4)  # 输出卷积结��


tensor([[[[ 1,  3,  4, 10,  8],
          [ 5, 10, 12, 12,  6],
          [ 7, 18, 16, 16,  8],
          [11, 13,  9,  3,  4],
          [14, 13,  9,  7,  4]]]])
